## import module

In [2]:
import pandas as pd
import numpy as np
import cvxpy as cp
from datetime import datetime, timedelta
import pandas_datareader.data as web

from dateutil.relativedelta import relativedelta

저장된 파일을 불러옵니다.

In [3]:
all_df = pd.read_csv('all_asset_2010_2020.csv')
all_df['Date'] = all_df.Date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
all_df.set_index('Date',inplace=True)

In [4]:
symbols = ['AAPL', 'MSFT', 'AMZN']# 'FB', 'GOOGL', 'GOOG', 'TSLA', 'NVDA', 'CMCSA', 'ADBE']
st ='2020-06-01'
en = '2020-10-30'
unittime = '1m'
lookback = '6m'
rebalance = '2m'

이후 날짜 설정을 용이하게 하기 위해 datetime으로 변경하는 부분입니다.

In [5]:
st = datetime(int(st[:4]),int(st[5:7]),int(st[8:10]))
en = datetime(int(en[:4]),int(en[5:7]),int(en[8:10]))
def timestr_to_datetime(s):
  if s[-1] == 'd':
    return timedelta(days=int(s[:-1]))
  elif s[-1] == 'w':
    return timedelta(weeks=int(s[:-1]))
  elif s[-1] == 'm':
    return relativedelta(months=int(s[:-1]))
  elif s[-1] == 'y':
    return relativedelta(years=int(s[:-1]))

unittime = timestr_to_datetime(unittime)
lookback = timestr_to_datetime(lookback)
rebalance = timestr_to_datetime(rebalance)

In [6]:
def mv_opt(data,symbols, cons=[''], ret_target = None): 
    ## 받아야 하는 걸로 symbols 추가함):
    n = len(data.T)   # asset number
    W = cp.Variable(n)  # weight of assets
    R = data.mean()*252
    C = data.cov()*252

    objective = cp.Minimize(cp.quad_form(W, C))
    constraints = [cp.sum(W) == 1, W >= 0]
    if 'n-sh' in cons:
        if ret_target == None:
            ret_target = np.dot(R.T, mv_opt(data, symbols)/100)
            ret_target = np.round(ret_target,2) + 0.02
        constraints.append(W @ R >= ret_target)
        
    if 'aloc-rst' in cons:
        STOP = False ## 여러 개 추가 가능, while 문으로 받기
        while not STOP:
            L = 0; H = 0 # 초기값
            L = int(input("Low bound(없으면 0 | 있으면 0.X , X에 해당하는 숫자만 입력)"))
            H = int(input("High bound(없으면 0 | 있으면 0.X , X에 해당하는 숫자만 입력)"))
            weight_symbol = input("제약 대상 주식 코드(띄어쓰기로 분리), 따옴표 X ").split(' ')
            Matrix = [0 for i in range(n)]
            temp= [symbols.index(i) for i in weight_symbol]
            for i in temp:
                Matrix[i] = 1 ## 제약 대상인 weight만 1로 남긴다. => sum(W @ Matrix)으로 H, L 바운드 설정

            if L & (not H): # Low bound만 있다. 
                constraints.append(cp.sum(W @ Matrix) >= L/10)        
            if (not L) & H: # High bound만 있다. 
                constraints.append(cp.sum(W @ Matrix) <= H/10)        
            if L & H: # 둘 다 있다. 
                constraints.append(cp.sum(W @ Matrix) >= L/10)        
                constraints.append(cp.sum(W @ Matrix) <= H/10)                                       
            STOP = bool(input("제약 추가 종료하겠습니까? True or False"))

    if 'Tracking-error' in cons:
        SP500 = web.DataReader(['sp500'], 'fred', st, en)
        sp_ret = sp_ret[index-lb:index].pct_change().dropna()

                    

        
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.ECOS)
    return W.value.round(6)*100

In [7]:
def backtest(symbols, st, en, ut,lb, rb,dropna=True):
  def data_by_dates():
    data = all_df[symbols]
    df = pd.DataFrame(columns = symbols)
    date = st-lookback
    while date < en:
        df.loc[data[date:].index[0]] = data[date:].values[0]
        date += ut
    rb_date = []
    date = st
    while date < en:
        rb_date.append(data[date:].index[0])
        date += rb
    return df, rb_date
  df, rb_date = data_by_dates()
  df_ret = df.pct_change().dropna()
  
  col = [i+'_amt' for i in symbols]
  col.append('total_amt')
  col.append('portfolio_ret')
  col+= [i+'_weight' for i in symbols]
  result = pd.DataFrame(columns = col)
  T_amt = 100

  for index, row in df.iterrows():
    if st <= index:
      if index in rb_date:
        weight = mv_opt(df_ret[index-lb:index], symbols) ## symbols 추가함
      amt = (df_ret[index:index]+1)*weight*T_amt/100
      for i in range(len(symbols)):
        s = symbols[i]
        result.loc[index,s+'_amt'] = amt[s].values[0]
        result.loc[index,s+'_weight'] = weight[i]
      T_amt_ = T_amt
      T_amt = amt.sum(axis = 1).values[0]
      result.loc[index,'total_amt'] = T_amt
      result.loc[index,'portfolio_ret'] = T_amt/T_amt_ - 1
  if dropna == True:
    return result.loc[:,result.eq(0).sum(axis=0) != len(result)]
  return result

In [8]:
df = backtest(symbols, st, en, unittime, lookback, rebalance)

In [9]:
import plotly.graph_objects as go

In [10]:
df

,MSFT_amt,AMZN_amt,total_amt,portfolio_ret,MSFT_weight,AMZN_weight
2020-06-01,13.0429,94.6685,107.711,0.0771142,12.4191,87.5809
2020-07-01,14.9769,109.897,124.874,0.159342,12.4191,87.5809
2020-08-03,40.9102,93.1838,134.094,0.0738313,30.9698,69.0302
2020-09-01,43.6918,104.084,147.776,0.10203,30.9698,69.0302
2020-10-01,109.969,27.7479,137.717,-0.0680701,79.6033,20.3967


# Back Test

In [28]:
## sp500의 monthly data (from yahoo)
sp = pd.read_csv('sp500_monthly.csv')
sp['sp_ret'] = sp['Adj Close'].pct_change()


In [29]:
sp.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sp_ret
0,2011-01-01,1257.619995,1302.670044,1257.619995,1286.119995,1286.119995,92164940000,NaN
1,2011-02-01,1289.140015,1344.069946,1289.140015,1327.219971,1327.219971,59223660000,0.031957
2,2011-03-01,1328.640015,1332.280029,1249.050049,1325.829956,1325.829956,89507640000,-0.001047
3,2011-04-01,1329.479980,1364.560059,1294.699951,1363.609985,1363.609985,77364810000,0.028495
4,2011-05-01,1365.209961,1370.579956,1311.800049,1345.199951,1345.199951,81708980000,-0.013501


In [30]:
## df 데이터의 date와 sp500의 data를 통일 시켜주기 위해 YYYY-DD-01 로 맞춤
df['year'] = df.index.year.astype(str)
df['month'] = df.index.month.astype(str)
df['month'] = df['month'].apply(lambda x: '0'+x if len(x) == 1 else x)
df['dateInt']=df['year'].astype(str) + '-' +df['month'].astype(str) + '-01'

In [31]:
## sp500데이터에서 df와 동일한 기간만을 추출
sp = sp[sp['Date'].isin(df['dateInt'])]

In [32]:
sp

,Date,Open,High,Low,Close,Adj Close,Volume,sp_ret
113,2020-06-01,3038.780029,3233.129883,2965.659912,3100.290039,3100.290039,131044000000,0.018388
114,2020-07-01,3105.919922,3279.989990,3101.169922,3271.120117,3271.120117,97197020000,0.055101
115,2020-08-01,3288.260010,3514.770020,3284.530029,3500.310059,3500.310059,84402300000,0.070065
116,2020-09-01,3507.439941,3588.110107,3209.449951,3363.000000,3363.000000,92084120000,-0.039228
117,2020-10-01,3385.870117,3549.850098,3233.939941,3269.959961,3269.959961,89737600000,-0.027666


## 수익률 차트

In [33]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.dateInt, y=df['portfolio_ret'],
                    mode='lines+markers',
                    name='Our_ret'))
fig.add_trace(go.Scatter(x=sp.Date, y=sp['sp_ret'],
                    mode='lines+markers',
                    name='SP500_ret'))

fig.show()

In [84]:
pf_ret =df['portfolio_ret'].values
sp_ret = sp['sp_ret'].values

In [85]:
from scipy.stats import norm

In [110]:
import math
def get_mdd(arr_v):
    """
    MDD(Maximum Draw-Down)
    :return: (mdd rate)
    """
#     arr_v = np.array(x)
    peak_lower = np.argmax(np.maximum.accumulate(arr_v) - arr_v)
    peak_upper = np.argmax(arr_v[:peak_lower])
    return (arr_v[peak_lower] - arr_v[peak_upper]) / arr_v[peak_upper]

# 출처 : http://blog.quantylab.com/mdd.html

In [105]:
Annual_return = []
Annual_variance = []
sharpe_ratio = []
VaR_95 = []
MDD = []

for data in [pf_ret, sp_ret]:
    mean = data.mean()
    std = data.std()
        
    Annual_return.append(mean * 12)
    Annual_variance.append(std* math.sqrt(12))
    sharpe_ratio.append(mean/std)
    VaR_95.append(norm.ppf(1-0.9, mean, std))
    MDD.append(get_mdd(data))
    

In [106]:
result = pd.DataFrame(index = ['portfolio', 'SP500'])

In [107]:
result['Annual_return'] = Annual_return
result['Annual_variance'] = Annual_variance
result['sharpe'] = sharpe_ratio
result['VaR_95'] = VaR_95
result['MDD'] = MDD

## Statistic result

In [108]:
result

,Annual_return,Annual_variance,sharpe,VaR_95,MDD
portfolio,0.826195,0.259831,0.917913,-0.027275,-1.427194
SP500,0.183986,0.150297,0.353380,-0.040271,-1.559882
